In [1]:
import bs4 as bs
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests
import numpy as np

#Einlesen der entsprechenden Excel-Dateien zur Vorbereitung der Arbeitslosendaten:

U1932 = pd.read_excel('1932' + '.xlsx', 'Gemeinde')
U1933 = pd.read_excel('1933' + '.xlsx', 'Gemeinde')
U1934 = pd.read_excel('1934' + '.xlsx', 'Gemeinde')
U1935 = pd.read_excel('1935' + '.xlsx', 'Gemeinde')
U1936 = pd.read_excel('1936' + '.xlsx', 'Gemeinde')

#Anpassen der Spaltenüberschriften, so dass diese einheitlich sind:

U1932.rename(columns={'Notstandsarbeiter_fremdebezirke_Insg.':'Notstandsarbeiter_fremdeBezirke_Insg.'}, inplace=True)
U1932.rename(columns={'Notstandsarbeiter_fremdebezirke':'Notstandsarbeiter_fremdeBezirke'}, inplace=True)
U1936.rename(columns={'Notstandsarbeiter_Insg.':'Notstandsarbeiter_Ings.'}, inplace=True)

#Zusammenfügen der unterschiedlichen Datensätze:

frames = [U1933,U1934,U1935,U1936]
result = U1932.append(frames)



In [2]:
#Anpassen der Spalten Monat und Jahr auf englische Begriffe, um im nächsten Schritt eine einheitliche Spalte zu definieren 
#mit der Datumsangabe:

result = result.rename(columns={"Jahr": "YEAR", "Monat": "MONTH"})
result['MONTH'] = result['MONTH'].replace({'Januar':'January', 'Februar':'February','März':'March', 'Mai':'May','Juni':'June', 'Juli':'July','Oktober':'October', 'Dezember':'December', 'February ':'February'})
result['MONTH'] = pd.to_datetime(result.MONTH, format='%B').dt.month
result['DATE'] = pd.to_datetime(result[['YEAR', 'MONTH']].assign(DAY=1))

#Sortieren der Zeilen alphabetisch nach 'Gemeinde' und zweitens nach dem Datum:

result = result.sort_values(['Gemeinde', 'DATE'])

#Lineare Interpolation, um die Spalte "Bevölkerung" zu vervollständigen:

result['Bevoelkerung'] = result.groupby('Gemeinde')['Bevoelkerung'].apply(lambda x : x.interpolate(method = "linear", order = 1, limit_direction = "both")).round(decimals=0)


In [3]:
#Setzen der Spalte "DATE" als Datum:

result.index = result['DATE']

#Entfernen der Spalten "Numid" und "DATE":

result = result.drop('Numid', 1)
result = result.drop('DATE', 1)

In [4]:
#Datensatz auf den Monat Februar beschränken:

result = result[result['MONTH'] == 2]

In [5]:
#Berechnung der Spalte "tot_unt_Arbeitslose_Einw", um die Arbeitslosenzahlen pro 1000 Einwohner zu berechnen:

result['Arbeitslosenversicherung_Einw'] = result['Arbeitslosenversicherung']/result['Bevoelkerung']*1000
result['Krisenfuersorge_Einw'] = result['Krisenfuersorge']/result['Bevoelkerung']*1000
result['tot_unt_Arbeitslose'] = result['Arbeitslosenversicherung'] + result['Krisenfuersorge']
result['tot_unt_Arbeitslose_Einw'] = result['tot_unt_Arbeitslose']/result['Bevoelkerung']*1000

In [6]:
#Entfernen aller Gemeinden, welche in der Spalte "tot_unt_Arbeitslose_Einw" keine Beobachtung ausweisen:

result['tot_unt_Arbeitslose_Einw'] = result['tot_unt_Arbeitslose_Einw'].fillna('')
result = result.groupby('Gemeinde').filter(lambda g: (g.tot_unt_Arbeitslose_Einw != '').all())

#Entfernen aller Gemeinden, welche weniger als fünf Zeilen ausweisen. 
#Dadurch wird geährleistet, dass alle Gemeinden definitiv Beobachtungen für die Jahre 1932 - 1936 ausweisen.

result = result.groupby('Gemeinde').filter(lambda x: len(x) > 4)

#result.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\Zusammenfassung Gemeinde.xlsx', index = True)

In [7]:
result1 = result[['Gemeinde','YEAR','Bevoelkerung','tot_unt_Arbeitslose','Arbeitslosenversicherung','Krisenfuersorge','tot_unt_Arbeitslose_Einw',
                 'Arbeitslosenversicherung_Einw','Krisenfuersorge_Einw']]

result1 = result1.loc[((result1['YEAR']==1932)| (result1['YEAR']==1936))]

#result1.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Bilder\Arbeitslosendaten.xlsx', index = True)
result1.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\Arbeitslosendaten.csv', index = True)
print(result1.head())

           Gemeinde  YEAR  Bevoelkerung  tot_unt_Arbeitslose  \
DATE                                                           
1932-02-01   Aachen  1932      154725.0               9190.0   
1936-02-01   Aachen  1936      162439.0               7427.0   
1932-02-01    Aalen  1932       12182.0                642.0   
1936-02-01    Aalen  1936       13056.0                 35.0   
1932-02-01    Ahlen  1932       22347.0               1131.0   

            Arbeitslosenversicherung  Krisenfuersorge  \
DATE                                                    
1932-02-01                    4306.0           4884.0   
1936-02-01                    1331.0           6096.0   
1932-02-01                     296.0            346.0   
1936-02-01                      12.0             23.0   
1932-02-01                     462.0            669.0   

           tot_unt_Arbeitslose_Einw  Arbeitslosenversicherung_Einw  \
DATE                                                                 
1932-02-01 

In [8]:
#################################################################################################################################
### ACHTUNG: hier wurden die kumulierten Flugzeugdaten verwendet ###
### weiter unten an der bezeichneten Stelle wird mit den jährlichen Daten gearbeitet.
### Allerdings werden Definitionen aus dieser alten Programmierung übernommen.

#Im folgenden wird der Datensatz zur Erstellung der großen Paneldatenanalyse erstellt

#Aus dem kompletten Arbeitslosen Datensatz werden die Gemeinden mit Flugzeugproduktion herausgefiltert:

rpanel = result[result['Gemeinde'].str.contains("Augsburg|Berlin|Brandenburg|Braunschweig|Bremen|Stuttgart|Dessau|Friedrichshafen|Gotha|Halle|Hamburg|Kassel|Leipzig|München|Oranienburg|Oschersleben|Regensburg|Rostock|Rosdock-Warnemünde|Wismar")]
rpanel = rpanel.loc[~((rpanel['Gemeinde']=='München-Gladbach-Rheydt')| (rpanel['Gemeinde']=='Wittenberg, Bz. Halle'))]

#Beschränken des Datensatzes auf die Spalten "Gemeinde", "YEAR", "MONTH", "tot_unt_Arbeitslose_Einw":

rpanel = rpanel[['Gemeinde', 'YEAR', 'MONTH', 'tot_unt_Arbeitslose_Einw']]

#Erstellen einer Spalte, welche analog zu den Jahreszahlen den Zeilen numerische Werte von 1-5 zuordnet:

rpanel['t'] = pd.Series(np.tile(['1', '2', '3', '4', '5'], rpanel['Gemeinde'].nunique()), index=rpanel.index) 

#Erstellen einer Spalte, welche analog zu der "Gemeinde"-Spalte den einzelnen Gemeinden numerische Werte zuordnet:

rpanel['i']=pd.Series(np.arange(rpanel['Gemeinde'].nunique()*5) // 5, index=rpanel.index)

#Einspielen des Flugzeugdatensatzes:

aircraft = pd.read_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Felix Fikowski Bachelor Thesis\cum aircraftprices by city' + '.xlsx','Sheet1')

#Beschränken des Datensatzes auf die Jahre 1932 - 1936:

aircraft = aircraft.iloc[ 2:7 , : ]

#Entfernen der Spalte "Zeit":

aircraft = aircraft.drop('Zeit',1)

#Hinzufügen der Flugzeugproduktionszahlen zu den Arbeitslosendaten. Dabei werden die Flugzeugproduktionsdaten noch passend angeordnet:

rpanel['aircraft'] = aircraft.T.values.reshape(19*5,1)

#Die München wird von diesem Datensatz entfernt, da im beobachteten Zeitraum in München keine Flugzeugproduktion stattfand:

rpanel = rpanel[rpanel.Gemeinde != 'München']

#rpanel.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpanel.csv', index = True)

In [10]:
#Im folgenden werden dem Paneldatensatz, welcher aktuell nur die Daten der Gemeinden mit Flugzeugproduktion enthält, auch die restlichen Gemeinden
#zugeordnet:

#Hierzu wird zunächst der Große Arbeitslosendatensatz herangezogen. 
#Dieser wird  auf die Spalten "Gemeinde", "YEAR", "MONTH", "tot_unt_Arbeitslose_Einw" reduziert:

rpanel2 = result[['Gemeinde', 'YEAR', 'MONTH', 'tot_unt_Arbeitslose_Einw']]

#Anschließend werden die Städte, welche Flugzeugproduktion hatten entfernt:

rpanel2 = rpanel2.loc[~((rpanel2['Gemeinde']=='Augsburg')| (rpanel2['Gemeinde']=='Berlin')| (rpanel2['Gemeinde']=='Brandenburg')| (rpanel2['Gemeinde']=='Braunschweig')| (rpanel2['Gemeinde']=='Bremen')| (rpanel2['Gemeinde']=='Dessau')| (rpanel2['Gemeinde']=='Friedrichshafen')| (rpanel2['Gemeinde']=='Gotha')| (rpanel2['Gemeinde']=='Halle a. d. Saale')| (rpanel2['Gemeinde']=='Hamburg')| (rpanel2['Gemeinde']=='Kassel')| (rpanel2['Gemeinde']=='Leipzig')| (rpanel2['Gemeinde']=='Oranienburg')| (rpanel2['Gemeinde']=='Oschersleben')| (rpanel2['Gemeinde']=='Regensburg')| (rpanel2['Gemeinde']=='Rostock')| (rpanel2['Gemeinde']=='Stuttgart')| (rpanel2['Gemeinde']=='Wismar'))]

#Nun kann man dem Datensatz die Spalte "aircraft" hinzufügen und den Wert überall auf null setzen:

rpanel2['aircraft'] = 0
rpanel2['aircraft'] = rpanel2['aircraft'].round(2)

#Der Paneldatensatz der Städte mit Flugzeugproduktion wird auf die gleichen Spalten beschränkt:

rpanel = rpanel[['Gemeinde', 'YEAR', 'MONTH', 'tot_unt_Arbeitslose_Einw', 'aircraft']]

#Abschließend können beide Datensätze zusammengefügt werden zum finalen, großen Paneldatensatz:

rpanel2 = rpanel.append(rpanel2)
print(rpanel2.head(50))
rpanel2.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpaneltotal.xlsx', index = True)
###############################################################################################################################################################################################################################

                     Gemeinde  YEAR  MONTH tot_unt_Arbeitslose_Einw  \
DATE                                                                  
1932-02-01           Augsburg  1932      2                  45.1962   
1933-02-01           Augsburg  1933      2                  31.1552   
1934-02-01           Augsburg  1934      2                  18.8391   
1935-02-01           Augsburg  1935      2                  17.2005   
1936-02-01           Augsburg  1936      2                  16.5995   
1932-02-01             Berlin  1932      2                  68.3973   
1933-02-01             Berlin  1933      2                  58.8874   
1934-02-01             Berlin  1934      2                  46.2358   
1935-02-01             Berlin  1935      2                  33.6221   
1936-02-01             Berlin  1936      2                  31.1968   
1932-02-01        Brandenburg  1932      2                  107.204   
1933-02-01        Brandenburg  1933      2                  56.5247   
1934-0

In [9]:
### Ab hier werden die jährlichen Daten genutzt, um den großen Datensatz zu erzeugen.
### Es wird genauso gearbeitet wie mit den kumulierten Daten

rpanel1 = rpanel[['Gemeinde', 'YEAR', 'MONTH', 'tot_unt_Arbeitslose_Einw']]
aircraft1 = pd.read_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Bilder\Flugzeugproduktionjährlich' + '.xlsx','Sheet1')
aircraft1 = aircraft1.loc[:, ~aircraft1.columns.str.contains('^Unnamed')]
rpanel1['aircraft'] = aircraft1.T.values.reshape(18*5,1)

#print(rpanel1.head(20))
#rpanel3 = rpanel1.append(rpanel2)
#print(rpanel3.head(50))

                Gemeinde  YEAR  MONTH tot_unt_Arbeitslose_Einw  aircraft
DATE                                                                    
1932-02-01      Augsburg  1932      2                  45.1962    469.10
1933-02-01      Augsburg  1933      2                  31.1552   4012.57
1934-02-01      Augsburg  1934      2                  18.8391   5265.55
1935-02-01      Augsburg  1935      2                  17.2005   3853.68
1936-02-01      Augsburg  1936      2                  16.5995   2528.23
1932-02-01        Berlin  1932      2                  68.3973      0.00
1933-02-01        Berlin  1933      2                  58.8874     12.18
1934-02-01        Berlin  1934      2                  46.2358    280.71
1935-02-01        Berlin  1935      2                  33.6221   1443.92
1936-02-01        Berlin  1936      2                  31.1968   3582.97
1932-02-01   Brandenburg  1932      2                  107.204     12.98
1933-02-01   Brandenburg  1933      2              

C:\Users\Felix\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
rpanel3 = result[['Gemeinde', 'YEAR', 'MONTH', 'tot_unt_Arbeitslose_Einw']]

rpanel3 = rpanel3.loc[~((rpanel3['Gemeinde']=='Augsburg')| (rpanel3['Gemeinde']=='Berlin')| (rpanel3['Gemeinde']=='Brandenburg')| (rpanel3['Gemeinde']=='Braunschweig')| (rpanel3['Gemeinde']=='Bremen')| (rpanel3['Gemeinde']=='Dessau')| (rpanel3['Gemeinde']=='Friedrichshafen')| (rpanel3['Gemeinde']=='Gotha')| (rpanel3['Gemeinde']=='Halle a. d. Saale')| (rpanel3['Gemeinde']=='Hamburg')| (rpanel3['Gemeinde']=='Kassel')| (rpanel3['Gemeinde']=='Leipzig')| (rpanel3['Gemeinde']=='Oranienburg')| (rpanel3['Gemeinde']=='Oschersleben')| (rpanel3['Gemeinde']=='Regensburg')| (rpanel3['Gemeinde']=='Rostock')| (rpanel3['Gemeinde']=='Stuttgart')| (rpanel3['Gemeinde']=='Wismar'))]

rpanel3['aircraft'] = 0
rpanel3['aircraft'] = rpanel3['aircraft'].round(2)

rpanel1 = rpanel1[['Gemeinde', 'YEAR', 'MONTH', 'tot_unt_Arbeitslose_Einw', 'aircraft']]

rpanel3 = rpanel1.append(rpanel3)

print(rpanel3.shape)

rpanel3.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpanelyearly.csv', index = True)
rpanel3.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpanelyearly.xlsx', index = True)

(2465, 5)


In [26]:
#Erzeigen einer numerischen Nummer für Stata regressionen

rpanel3['i']=pd.Series(np.arange(rpanel3['Gemeinde'].nunique()*5) // 5, index=rpanel3.index)

print(rpanel3.head(20))

rpanel3.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpanelyearly3.csv', index = True)


                Gemeinde  YEAR  MONTH tot_unt_Arbeitslose_Einw  aircraft  i
DATE                                                                       
1932-02-01      Augsburg  1932      2                  45.1962    469.10  0
1933-02-01      Augsburg  1933      2                  31.1552   4012.57  0
1934-02-01      Augsburg  1934      2                  18.8391   5265.55  0
1935-02-01      Augsburg  1935      2                  17.2005   3853.68  0
1936-02-01      Augsburg  1936      2                  16.5995   2528.23  0
1932-02-01        Berlin  1932      2                  68.3973      0.00  1
1933-02-01        Berlin  1933      2                  58.8874     12.18  1
1934-02-01        Berlin  1934      2                  46.2358    280.71  1
1935-02-01        Berlin  1935      2                  33.6221   1443.92  1
1936-02-01        Berlin  1936      2                  31.1968   3582.97  1
1932-02-01   Brandenburg  1932      2                  107.204     12.98  2
1933-02-01  

In [27]:
#Im folgenden werden die in R gematchten Gemeinden aus dem großen Paneldatensatz ermittelt

#Einspielen der beiden Datensätze. Der über R generierte Datensatz, der die gematchten Gemeinden enthält und der Datensatz der die kompletten Paneldaten enthält:

match = pd.read_csv('match' + '.csv')
rpanelyearly = pd.read_csv('rpanelyearly3' + '.csv')

#Umformatieren der Gemeindenamen im Paneldatensatz, so dass diese identisch sind mit den Namen im gematchten Datensatz:

rpanelyearly['Gemeinde'] = rpanelyearly['Gemeinde'].str.upper()
rpanelyearly['Gemeinde'] = rpanelyearly["Gemeinde"].str.replace("Ü", "UE")
rpanelyearly['Gemeinde'] = rpanelyearly["Gemeinde"].str.replace("Ö", "OE")
rpanelyearly['Gemeinde'] = rpanelyearly["Gemeinde"].str.replace("Ä", "AE")

#Erstellen einer Liste die die gematchten Gemeindenamen enthält:

match_name = match['name'].unique()

#Mithilfe dieser Liste wird nun eine Formel erstellt, welche den gematchten Gemeinden in der neuen Spalte "com" den Wert 1 zuweist und allen anderen den Wert 0:

def g(row):
    if pd.np.isin(row['Gemeinde'], match_name) == True:
        val = 1
    else:
        val = 0
    return val

#Anwendung der Formel:

rpanelyearly['com'] = rpanelyearly.apply(g, axis=1)

#Die Daten der Gemeinde Stuttgart werden ebenfalls mit dem Wert 1 versehen, da Stuttgart Böblingen approximiert:

rpanelyearly.loc[rpanelyearly.Gemeinde == 'STUTTGART', 'com'] = 1

#Filtern des auf die Daten, denen in der Spalte "com" der Wert 1 zugeordnet wurde:

rpanelyearly = rpanelyearly[rpanelyearly['com'] == 1]
print(rpanelyearly.shape)
rpanelyearly.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpanelyearly2.csv', index = True)

(320, 8)


In [14]:
###########################################################################################################
### Achtung wieder kumuliert ###

#Im folgenden werden die in R gematchten Gemeinden aus dem großen Paneldatensatz ermittelt

#Einspielen der beiden Datensätze. Der über R generierte Datensatz, der die gematchten Gemeinden enthält und der Datensatz der die kompletten Paneldaten enthält:

match = pd.read_csv('match' + '.csv')
rpanelfinal = pd.read_excel('rpaneltotal' + '.xlsx', 'Sheet1')

#Umformatieren der Gemeindenamen im Paneldatensatz, so dass diese identisch sind mit den Namen im gematchten Datensatz:

rpanelfinal['Gemeinde'] = rpanelfinal['Gemeinde'].str.upper()
rpanelfinal['Gemeinde'] = rpanelfinal["Gemeinde"].str.replace("Ü", "UE")
rpanelfinal['Gemeinde'] = rpanelfinal["Gemeinde"].str.replace("Ö", "OE")
rpanelfinal['Gemeinde'] = rpanelfinal["Gemeinde"].str.replace("Ä", "AE")

#Erstellen einer Liste die die gematchten Gemeindenamen enthält:

match_name = match['name'].unique()

#Mithilfe dieser Liste wird nun eine Formel erstellt, welche den gematchten Gemeinden in der neuen Spalte "com" den Wert 1 zuweist und allen anderen den Wert 0:

def g(row):
    if pd.np.isin(row['Gemeinde'], match_name) == True:
        val = 1
    else:
        val = 0
    return val

#Anwendung der Formel:

rpanelfinal['com'] = rpanelfinal.apply(g, axis=1)

#Die Daten der Gemeinde Stuttgart werden ebenfalls mit dem Wert 1 versehen, da Stuttgart Böblingen approximiert:

rpanelfinal.loc[rpanelfinal.Gemeinde == 'STUTTGART', 'com'] = 1

#Filtern des auf die Daten, denen in der Spalte "com" der Wert 1 zugeordnet wurde:

rpanelfinal = rpanelfinal[rpanelfinal['com'] == 1]

#rpanelfinal.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpaneltotal2.csv', index = True)

#######################################################################################################

In [15]:
#Nachfolgend wird ein Datensatz für eine Diffrence-in-Difference-Analyse vorbereitet, welche jedoch keine Verwendung in der eigentlichen Arbeit fand.
#Da jedoch einzelne Elemente dieses Datensatzes verwendet werden, erfolgt hier nochmal eine genauere Erläuterung der Aufbereitungsschritte.

#Anhand der Buchstabenkombinationen werden die Städte mit Flugzeugproduktion herausgefilttert. 
#Dabei kommt es dazu, dass auch die Gemeinden München-Gladbach-Rheydt und Wittenberg, Bz. Halle selektiert werden.
#Diese werden wieder entfernt.

treatment = result[result['Gemeinde'].str.contains("Augsburg|Berlin|Brandenburg|Braunschweig|Bremen|Stuttgart|Dessau|Friedrichshafen|Gotha|Halle|Hamburg|Kassel|Leipzig|München|Oranienburg|Oschersleben|Regensburg|Rostock|Rosdock-Warnemünde|Wismar")]
treatment = treatment.loc[~((treatment['Gemeinde']=='München-Gladbach-Rheydt')| (treatment['Gemeinde']=='Wittenberg, Bz. Halle'))]

#Die einzelnen Gemeinden werden anschließend in eine eigene Liste geschrieben.

columns = treatment['Gemeinde'].unique().tolist()



index = treatment.index.unique()
treatment = treatment['tot_unt_Arbeitslose_Einw'].values.reshape(19,5)
treatment = pd.DataFrame(treatment, index = columns, columns = index)
treatment = treatment.T
del treatment.index.name
treatment = treatment.T

treatment['treatment'] = 1
cols = treatment.columns.tolist()
cols = cols[-1:] + cols[:-1]
treatment = treatment[cols]

#print(treatment)

In [16]:
nontreatment = result.loc[~((result['Gemeinde']=='Augsburg')| (result['Gemeinde']=='Berlin')| (result['Gemeinde']=='Brandenburg')| (result['Gemeinde']=='Braunschweig')| (result['Gemeinde']=='Bremen')| (result['Gemeinde']=='Dessau')| (result['Gemeinde']=='Friedrichshafen')| (result['Gemeinde']=='Gotha')| (result['Gemeinde']=='Halle a. d. Saale')| (result['Gemeinde']=='Hamburg')| (result['Gemeinde']=='Kassel')| (result['Gemeinde']=='Leipzig')| (result['Gemeinde']=='München')| (result['Gemeinde']=='Oranienburg')| (result['Gemeinde']=='Oschersleben')| (result['Gemeinde']=='Regensburg')| (result['Gemeinde']=='Rostock')| (result['Gemeinde']=='Stuttgart')| (result['Gemeinde']=='Wismar'))]

columns1 = nontreatment['Gemeinde'].unique().tolist()
index = nontreatment.index.unique()
nontreatment = nontreatment['tot_unt_Arbeitslose_Einw'].values.reshape(nontreatment['Gemeinde'].nunique(),5)
nontreatment = pd.DataFrame(nontreatment, index = columns1, columns = index)
nontreatment = nontreatment.T
del nontreatment.index.name
nontreatment = nontreatment.T

nontreatment['treatment'] = 0
cols = nontreatment.columns.tolist()
cols = cols[-1:] + cols[:-1]
nontreatment = nontreatment[cols]

#print(nontreatment.head(50))

In [17]:
dataframe = treatment.append(nontreatment)

#print(dataframe.shape)
#print(dataframe.head(50))
#dataframe.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\unemployment Gemeinde.xlsx', index = True)

In [18]:
#Nun wird der eigentliche Datensatz zur Diff.-in-Diff.-Schätzung erstellt.
#Aus dem Datensatz, welcher alle Arbeitslosenzahlen vollumfänglich enthält werden nur die Spalten Gemeinde, YEAR, MONTH und tot_unt_Arbeitslose_Einw selektiert.
#Außerdem wurden nur die Daten für die Jahre 1932 und 1936 herangezogen.

did = result[['Gemeinde', 'YEAR', 'MONTH', 'tot_unt_Arbeitslose_Einw']]
did = did.loc[((did['YEAR'] == 1932)| (did['YEAR'] == 1936))]

#Anschließend wird wieder eine Formel definiert, die anhand der Liste, die die Namen der Gemeinden mit Flugzeugproduktion enthält,
#den Gemeinden danach ob Sie in dieser Liste sind den Wert 1 oder 0 zuordnet.

def f(row):
    if pd.np.isin(row['Gemeinde'], columns) == True:
        val = 1
    else:
        val = 0
    return val

#Diese Formel wird auf den Datensatz zur Diff.-in-Diff.-Analyse angewendet und es wird die Spalte 'treatment' erstellt.

did['treatment'] = did.apply(f, axis=1)

#did.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\did.xlsx')
#did.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\did.csv')
gemeinde = did['Gemeinde'].unique()

In [19]:
#Nachfolgend wird die Datenbank "Wahl- und Sozialdaten der Kreise und Gemeinden des Deutschen Reiches von 1920 bis 1933" von Dirk Hänisch aufbereitet.
#Hierzu müssen zuerst die Gemeinden gefiltert werden die auch im unserem Datensatz zu den Arbeitslosenzahlen vorhanden sind.

#Aus dem Diff.-in-Diff.-Datensatz werden nur die Spalten 'treatment' und 'Gemeinde' benötigt:

pre = did[['Gemeinde', 'treatment']]

#Einlesen der Sozialdaten und anschließende Sortierung nach dem Namen der Gemeinden/Kreise.

social = pd.read_stata('ZA8013_Sozialdaten' + '.dta')
social = social.sort_values(['name'])

#Nun müssen die Textzeichen der betreffenden Einheiten, in dem Fall Gemeinden und Kreise, vergleichbar gemacht werden.

#Als erstes werden die Daten aus dem Diff.-in-Diff.-Datensatz angepasst, indem die Buchstaben der Spalte Gemeinde groß geschrieben werde.
#Zusätzlich werden Umlaute ausgetauscht.

pre['Gemeinde'] = pre['Gemeinde'].str.upper()

pre['Gemeinde'] = pre["Gemeinde"].str.replace("Ü", "UE")
pre['Gemeinde'] = pre["Gemeinde"].str.replace("Ö", "OE")
pre['Gemeinde'] = pre["Gemeinde"].str.replace("Ä", "AE")

#Es wird eine Liste erstellt die alle Gemeindenamen aus dem vollständigen Datensatz zu den Arbeitslosenzahlen enthält.

gemeinde = pre['Gemeinde'].unique()

#Bei den Sozialdaten müssen die Buchstaben ebenfalls angepasst werden, so dass überflüssige Leerzeichen und Anhängsel entfert werden

social['name'] = social["name"].str.replace(" L", "")
social['name'] = social["name"].str.replace(" S", "")
social['name'] = social['name'].str.rstrip()

#Im folgenden werden alle weiterhin abweichenden Namen, welche nicht systematisch abweichen, in den Sozialdaten ebenfalls geändert.

social['name'] = social['name'].replace({'BERNAU BEI BERLIN':'BERNAU', 'BRANDENBURG A. H.':'BRANDENBURG',
                                         'BURG B. M.':'BURG', 'DUISBURG (-31.7.29)':'DUISBURG','FREIENWALDE A. O. (BAD)':'FREIENWALDE', 
                                         'GODESBERG  (BAD)':'GODESBERG','HAMBURG MIT HAFEN':'HAMBURG', 'HOMBERG (NIEDERRH.)':'HOMBERG', 
                                         'KREFELD-UERDINGEN  (RHEIN':'KREFELD-UERDINGEN', 'LAUENBURG (POMMERN)':'LAUENBURG', 
                                         'LAUENBURG I. POM.':'LAUENBURG', 'LENGERICH  G.':'LENGERICH', 'LENGERICH  T.':'LENGERICH', 
                                         'LENGERICH (ST+LG)':'LENGERICH', 'MARIENBURG (WESTPR.)':'MARIENBURG', 'MARIENBURG I. WESTPR.':'MARIENBURG',
                                         'NAUMBURG (SAALE) (-30.':'NAUENBURG', 'NAUMBURG (SAALE)':'NAUENBURG', 'OSTERODE I. OSTPR.':'OSTERODE', 
                                         'OSTERODE I. OSTPR.':'OSTERODE', 'RHEINE RECHTS D. EMS':'RHEINE', 'HALLE (SAALE)':'HALLE A. D. SAALE',
                                         'AUERBACH':'AUERBACH I. VOGTLD.', 'BERGISCH GLADBACH':'BERG.-GLADBACH', 'CALBE':'CALBE A. D. SAALE', 
                                         'CALBE A..':'CALBE A. D. SAALE', 'CUESTRIN (1928- KUESTRIN)':'KUESTRIN', 'DEUTSCH EYLAU':'DEUTSCH-EYLAU', 
                                         'DEUTSCH KRONE':'DEUTSCH-KRONE', 'FORST (LAUSITZ)':'FORST I. D. LAUSITZ', 'FRANKFURT':'FRANKFURT A. M.', 
                                         'FRANKFURT (ODER)':'FRANKFURT A. D. O.', 'FUERSTENWALDE (A.D.PREE)':'FUERSTENWALDE', 'GEISLINGEN':'GEISLINGEN (STEIGE)', 
                                         'GEISLINGEN (-24, GEISLINGENTEIGE)':'GEISLINGEN (STEIGE)', 'GLADBACH-RHEYDT':'MUENCHEN-GLADBACH-RHEYDT', 
                                         'GROSS OTTERSLEBEN':'GROSS-OTTERSLEBEN', 'HALBERSTADT (-30.9.32)':'NORDHARZ (HALBERSTADT)', 
                                         'HALBERSTADT':'NORDHARZ (HALBERSTADT)', 'HARBURG-WILHELMSBURG  (1.7.27-)':'HARBURG-WILHELMSBURG', 
                                         'KOENIGSBERG (NEUMARK)':'KOENIGSBERG', 'LANDAU (PFALZ)':'LANDAU I. D. PFALZ', 'LANDSBERG (WARTHE)':'LANDSBERG A. D. W.', 
                                         'LICHTENSTEIN-CALLENBERG (=.-CALLNBERG)':'LICHTENSTEIN-CALLNBERG', 'LUDWIGSHAFEN (RHEIN)':'LUDWIGSHAFEN A. RH.', 
                                         'MANSFELD':'MANSFELD (EISLEBEN)', 'MANSFELDER GEBIRGSKREIS':'MANSFELD (EISLEBEN)', 'MUEHLHAUSEN':'MUEHLHAUSEN I. THUER', 
                                         'MUELHEIM (RUHR)':'MUELHEIM A. D. RUHR', 'MUENDEN':'HANN. MUENDEN', 'MUENDEN (HANN. MUENDEN)':'HANN. MUENDEN', 
                                         'NAUENBURG':'NAUMBURG', 'NEUSALZ  (ODER)':'NEUSALZ A. D. O.', 'NEUSALZ A. O.':'NEUSALZ A. D. O.', 
                                         'NEUSTADT (HARDT)':'NEUSTADT (HAARDT)', 'NEUSTADT (O.S)':'NEUSTADT O. S.', 'NOWAWES':'POTSDAM-NOWAWES', 
                                         'NOWAWES -S':'POTSDAM-NOWAWES', 'OELSNITZ':'OELSNITZ I. VOGTLD.', 'OELSNITZ I.E.':'OELSNITZ (ERZGEB.)', 
                                         'PLAUEN':'PLAUEN I. VOGTLD.', 'REICHENBACH I. O.LAUS.':'REICHENBACH I. VOGTLD.', 'REICHENBACH':'REICHENBACH I. EULENGEB.', 
                                         'REST-NAUMBURG (SAALE) (-30.':'NAUMBURG', 'SCHOENEBECK A. E.':'SCHOENEBECK', 'SINGEN':'SINGEN (HOHENTWIEL)', 
                                         'STARGARD (POMMERN)':'STARGARD I. POM.', 'STENDAL':'ALTMARK (STENDAL)', 'STOLLBERG':'STOLLBERG (ERZGEB.)', 
                                         'ULM':'ULM A. D. DONAU', 'WEIDENAU':'WEIDENAU A. D. SIEG', 'WEISSTEIN':'WEISSSTEIN', 'WERNE A.IPPET.':'WERNE A. D. LIPPE', 
                                         'WILHELMSHAVEN':'WILHELMSHAVEN-RUESTR.', 'WITTENBERG':'WITTENBERG, BZ. HALLE','BERLIN-MITTE BA 1':'BERLIN'})    

#Die hier definierte Formel weißt jeder Gemeinde aus dem Datensatz der Arbeitslosenzahlen den Wert eins zu

def g(row):
    if pd.np.isin(row['name'], gemeinde) == True:
        val = 1
    else:
        val = 0
    return val

social['com'] = social.apply(g, axis=1)
social1 = social[social['com'] == 1]
social0 = social[social['com'] == 0]

soc = social1['name'].unique()
main_list = np.setdiff1d(gemeinde,soc)
social0 = social0[social0['name'].str.contains(r'\b(?:{})\b'.format('|'.join(main_list)))]
x = pd.Series(main_list) 

print(main_list)

#print(social1.head(60))
#print(social0.head(50))
#print(social['com'].value_counts())

#print(did.shape)
#print(gemeinde)
#print(socialue)
#print(social.shape)
print(social1['name'].nunique())
#print(did['Gemeinde'].nunique())
print(len(main_list))

print(social['name'].nunique())
print(social.shape)
#social.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\social.xlsx')
#x.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\x.xlsx')
#social0.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\social0.xlsx')
#social1.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\social1.xlsx')


C:\Users\Felix\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Felix\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Felix\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

['BADEN-BADEN' 'BELGARD A. D. PERS' 'LOCKSTEDT' 'SENSTENBERG']
489
4
4993
(6304, 114)


In [20]:
print(social1.shape)

frame1 = ['N327WB ','N327NSDA','N32NWB','N32NNSDA ','N333WB','N333NSDA']
frame2 = ['C25WOHN','C33POP1','C33POP1','C33ERWP','C33WERWP','C33INDU','C33BEAMT','C25POP','C25PROT','C25KATH','C25JUDEN']

(888, 114)


In [21]:
print(social1['agglvl'].value_counts())
social1 = social1.replace(-9, np.nan)

treatment1 = social1[social1['name'].str.contains("AUGSBURG|BERLIN|BRANDENBURG|BRAUNSCHWEIG|BREMEN|BOEBLINGEEN|DESSAU|FRIEDRICHSHAFEN|GOTHA|HALLE|HAMBURG|KASSEL|LEIPZIG|MUENCHEN|ORANIENBURG|OSCHERSLEBEN|REGENSBURG|ROSTOCK|WISMAR")]
treatment1 = treatment1.loc[~((treatment1['name']=='MUENCHEN-GLADBACH-RHEYDT')| (treatment1['name']=='WITTENBERG, BZ. HALLE')| (treatment1['name']=='NEUBRANDENBURG'))]

treatment1['treatment'] = 'treatment'

print(treatment1['name'].unique())

nontreatment1 = social1.loc[~((social1['name']=='AUGSBURG')| (social1['name']=='BERLIN')| (social1['name']=='BRANDENBURG')| (social1['name']=='BRAUNSCHWEIG')| (social1['name']=='BREMEN')| (social1['name']=='DESSAU')| (social1['name']=='FRIEDRICHSHAFEN')| (social1['name']=='GOTHA')| (social1['name']=='HALLE A. D. SAALE')| (social1['name']=='HAMBURG')| (social1['name']=='KASSEL')| (social1['name']=='LEIPZIG')| (social1['name']=='MUENCHEN')| (social1['name']=='ORANIENBURG')| (social1['name']=='OSCHERSLEBEN')| (social1['name']=='REGENSBURG')| (social1['name']=='ROSTOCK')| (social1['name']=='STUTTGART')| (social1['name']=='WISMAR'))]
nontreatment1['treatment'] = nontreatment

social2 = treatment1.append(nontreatment1)

print(social1.shape)
print(treatment1['name'].nunique())
print(nontreatment1['name'].nunique())
print(social2.shape)

print(social2.head(50))

GEMEINDEN AB 2000 E.    403
KREISE M.GEMEINDEN >    282
stadtkreise             194
KREISE O.GEMEINDEN >      8
RESTKREISE (GEM.< 20      1
Name: agglvl, dtype: int64
['AUGSBURG' 'BERLIN' 'BRANDENBURG' 'BRAUNSCHWEIG' 'BREMEN' 'DESSAU'
 'FRIEDRICHSHAFEN' 'GOTHA' 'HALLE A. D. SAALE' 'HAMBURG' 'KASSEL'
 'LEIPZIG' 'MUENCHEN' 'ORANIENBURG' 'OSCHERSLEBEN' 'REGENSBURG' 'ROSTOCK'
 'WISMAR']


ValueError: Wrong number of items passed 6, placement implies 1

In [ ]:
social9 = social2[(social2['agglvl'] == 'KREISE M.GEMEINDEN >')]
social4 = social2[(social2['agglvl'] == 'GEMEINDEN AB 2000 E.')]
social3 = social2[(social2['agglvl'] == 'stadtkreise')]
social5 = social2[(social2['agglvl'] == 'KREISE O.GEMEINDEN >')]
social6 = social2[(social2['agglvl'] == 'RESTKREISE (GEM.< 20')]

stadtkreise = social3['name'].unique().tolist()
gemeinde2000 = social4['name'].unique().tolist()
inter = set(stadtkreise).intersection(gemeinde2000)

social4 = social4[~social4['name'].isin(inter)]

social7 = social3.append(social4)

list1 = social7['name'].unique().tolist()
list2 = social5['name'].unique().tolist()

inter1 = set(list1).intersection(list2)

social5 = social5[~social5['name'].isin(inter1)]

social8 = social7.append([social5, social6])

list1 = social8['name'].unique().tolist()
list2 = social9['name'].unique().tolist()

inter2 = set(list1).intersection(list2)

social9 = social9[~social9['name'].isin(inter2)]

social10 = social8.append(social9)

vc = social10.name.value_counts()
vc = vc[vc>1].index.tolist()

social10.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\social10.csv')

In [ ]:
socialfinal = pd.read_excel('socialfinal' + '.xlsx', 'Sheet1')
print(socialfinal['name'].nunique())
print(socialfinal.shape)
socialfinal.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\socialfinal.csv')

wahlfinal = pd.read_excel('wahlfinal' + '.xlsx', 'Sheet1')
print(wahlfinal['name'].nunique())

wahlfinal.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\wahlfinal.csv')


In [ ]:
###### Analog wie zu den Sozialdaten werden die Wahldaten verarbeitet #####

wahl = pd.read_stata('ZA8013_Wahldaten' + '.dta')

wahl = wahl.sort_values(['name'])
wahlue = wahl.columns.to_list()

wahl = pd.read_stata('ZA8013_Wahldaten' + '.dta')
wahl = wahl.sort_values(['name'])

wahl['name'] = wahl["name"].str.replace(" L", "")
wahl['name'] = wahl["name"].str.replace(" S", "")
wahl['name'] = wahl['name'].str.rstrip()

wahl['name'] = wahl['name'].replace({'BERNAU BEI BERLIN':'BERNAU', 'BRANDENBURG A. H.':'BRANDENBURG',
                                         'BURG B. M.':'BURG', 'DUISBURG (-31.7.29)':'DUISBURG','FREIENWALDE A. O. (BAD)':'FREIENWALDE', 
                                         'GODESBERG  (BAD)':'GODESBERG','HAMBURG MIT HAFEN':'HAMBURG', 'HOMBERG (NIEDERRH.)':'HOMBERG', 
                                         'KREFELD-UERDINGEN  (RHEIN':'KREFELD-UERDINGEN', 'LAUENBURG (POMMERN)':'LAUENBURG', 
                                         'LAUENBURG I. POM.':'LAUENBURG', 'LENGERICH  G.':'LENGERICH', 'LENGERICH  T.':'LENGERICH', 
                                         'LENGERICH (ST+LG)':'LENGERICH', 'MARIENBURG (WESTPR.)':'MARIENBURG', 'MARIENBURG I. WESTPR.':'MARIENBURG',
                                         'NAUMBURG (SAALE) (-30.':'NAUENBURG', 'NAUMBURG (SAALE)':'NAUENBURG', 'OSTERODE I. OSTPR.':'OSTERODE', 
                                         'OSTERODE I. OSTPR.':'OSTERODE', 'RHEINE RECHTS D. EMS':'RHEINE', 'HALLE (SAALE)':'HALLE A. D. SAALE',
                                         'AUERBACH':'AUERBACH I. VOGTLD.', 'BERGISCH GLADBACH':'BERG.-GLADBACH', 'CALBE':'CALBE A. D. SAALE', 
                                         'CALBE A..':'CALBE A. D. SAALE', 'CUESTRIN (1928- KUESTRIN)':'KUESTRIN', 'DEUTSCH EYLAU':'DEUTSCH-EYLAU', 
                                         'DEUTSCH KRONE':'DEUTSCH-KRONE', 'FORST (LAUSITZ)':'FORST I. D. LAUSITZ', 'FRANKFURT':'FRANKFURT A. M.', 
                                         'FRANKFURT (ODER)':'FRANKFURT A. D. O.', 'FUERSTENWALDE (A.D.PREE)':'FUERSTENWALDE', 'GEISLINGEN':'GEISLINGEN (STEIGE)', 
                                         'GEISLINGEN (-24, GEISLINGENTEIGE)':'GEISLINGEN (STEIGE)', 'GLADBACH-RHEYDT':'MUENCHEN-GLADBACH-RHEYDT', 
                                         'GROSS OTTERSLEBEN':'GROSS-OTTERSLEBEN', 'HALBERSTADT (-30.9.32)':'NORDHARZ (HALBERSTADT)', 
                                         'HALBERSTADT':'NORDHARZ (HALBERSTADT)', 'HARBURG-WILHELMSBURG  (1.7.27-)':'HARBURG-WILHELMSBURG', 
                                         'KOENIGSBERG (NEUMARK)':'KOENIGSBERG', 'LANDAU (PFALZ)':'LANDAU I. D. PFALZ', 'LANDSBERG (WARTHE)':'LANDSBERG A. D. W.', 
                                         'LICHTENSTEIN-CALLENBERG (=.-CALLNBERG)':'LICHTENSTEIN-CALLNBERG', 'LUDWIGSHAFEN (RHEIN)':'LUDWIGSHAFEN A. RH.', 
                                         'MANSFELD':'MANSFELD (EISLEBEN)', 'MANSFELDER GEBIRGSKREIS':'MANSFELD (EISLEBEN)', 'MUEHLHAUSEN':'MUEHLHAUSEN I. THUER', 
                                         'MUELHEIM (RUHR)':'MUELHEIM A. D. RUHR', 'MUENDEN':'HANN. MUENDEN', 'MUENDEN (HANN. MUENDEN)':'HANN. MUENDEN', 
                                         'NAUENBURG':'NAUMBURG', 'NEUSALZ  (ODER)':'NEUSALZ A. D. O.', 'NEUSALZ A. O.':'NEUSALZ A. D. O.', 
                                         'NEUSTADT (HARDT)':'NEUSTADT (HAARDT)', 'NEUSTADT (O.S)':'NEUSTADT O. S.', 'NOWAWES':'POTSDAM-NOWAWES', 
                                         'NOWAWES -S':'POTSDAM-NOWAWES', 'OELSNITZ':'OELSNITZ I. VOGTLD.', 'OELSNITZ I.E.':'OELSNITZ (ERZGEB.)', 
                                         'PLAUEN':'PLAUEN I. VOGTLD.', 'REICHENBACH I. O.LAUS.':'REICHENBACH I. VOGTLD.', 'REICHENBACH':'REICHENBACH I. EULENGEB.', 
                                         'REST-NAUMBURG (SAALE) (-30.':'NAUMBURG', 'SCHOENEBECK A. E.':'SCHOENEBECK', 'SINGEN':'SINGEN (HOHENTWIEL)', 
                                         'STARGARD (POMMERN)':'STARGARD I. POM.', 'STENDAL':'ALTMARK (STENDAL)', 'STOLLBERG':'STOLLBERG (ERZGEB.)', 
                                         'ULM':'ULM A. D. DONAU', 'WEIDENAU':'WEIDENAU A. D. SIEG', 'WEISSTEIN':'WEISSSTEIN', 'WERNE A.IPPET.':'WERNE A. D. LIPPE', 
                                         'WILHELMSHAVEN':'WILHELMSHAVEN-RUESTR.', 'WITTENBERG':'WITTENBERG, BZ. HALLE','BERLIN-MITTE BA 1':'BERLIN'})    


def g(row):
    if pd.np.isin(row['name'], gemeinde) == True:
        val = 1
    else:
        val = 0
    return val

wahl['com'] = wahl.apply(g, axis=1)
wahl1 = wahl[wahl['com'] == 1]
wahl0 = wahl[wahl['com'] == 0]

soc = wahl1['name'].unique()
main_list = np.setdiff1d(gemeinde,soc)
social0 = social0[social0['name'].str.contains(r'\b(?:{})\b'.format('|'.join(main_list)))]
x = pd.Series(main_list) 

print(main_list)
wahl.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\wahl.xlsx')

print(wahl1['name'].nunique())
print(did['Gemeinde'].nunique())
print(len(main_list))

In [ ]:
wahl1 = wahl1.replace(-9, np.nan)

treatment2 = wahl1[wahl1['name'].str.contains("AUGSBURG|BERLIN|BRANDENBURG|BRAUNSCHWEIG|BREMEN|STUTTGART|DESSAU|FRIEDRICHSHAFEN|GOTHA|HALLE|HAMBURG|KASSEL|LEIPZIG|MUENCHEN|ORANIENBURG|OSCHERSLEBEN|REGENSBURG|ROSTOCK|WISMAR")]
treatment2 = treatment2.loc[~((treatment2['name']=='MUENCHEN-GLADBACH-RHEYDT')| (treatment2['name']=='WITTENBERG, BZ. HALLE')| (treatment2['name']=='NEUBRANDENBURG'))]

treatment2['treatment'] = 1

nontreatment2 = wahl1.loc[~((wahl1['name']=='AUGSBURG')| (wahl1['name']=='BERLIN')| (wahl1['name']=='BRANDENBURG')| (wahl1['name']=='BRAUNSCHWEIG')| (wahl1['name']=='BREMEN')| (wahl1['name']=='DESSAU')| (wahl1['name']=='FRIEDRICHSHAFEN')| (wahl1['name']=='GOTHA')| (wahl1['name']=='HALLE A. D. SAALE')| (wahl1['name']=='HAMBURG')| (wahl1['name']=='KASSEL')| (wahl1['name']=='LEIPZIG')| (wahl1['name']=='MUENCHEN')| (wahl1['name']=='ORANIENBURG')| (wahl1['name']=='OSCHERSLEBEN')| (wahl1['name']=='REGENSBURG')| (wahl1['name']=='ROSTOCK')| (wahl1['name']=='STUTTGART')| (wahl1['name']=='WISMAR'))]
nontreatment2['treatment'] = 0

wahl2 = treatment2.append(nontreatment2)

In [ ]:
wahl9 = wahl2[(wahl2['agglvl'] == 'KREISE M.GEMEINDEN >')]
wahl4 = wahl2[(wahl2['agglvl'] == 'GEMEINDEN AB 2000 E.')]
wahl3 = wahl2[(wahl2['agglvl'] == 'stadtkreise')]
wahl5 = wahl2[(wahl2['agglvl'] == 'KREISE O.GEMEINDEN >')]
wahl6 = wahl2[(wahl2['agglvl'] == 'RESTKREISE (GEM.< 20')]

stadtkreise = wahl3['name'].unique().tolist()
gemeinde2000 = wahl4['name'].unique().tolist()
inter = set(stadtkreise).intersection(gemeinde2000)

wahl4 = wahl4[~wahl4['name'].isin(inter)]

wahl7 = wahl3.append(wahl4)

list1 = wahl7['name'].unique().tolist()
list2 = wahl5['name'].unique().tolist()

inter1 = set(list1).intersection(list2)

wahl5 = wahl5[~wahl5['name'].isin(inter1)]

wahl8 = wahl7.append([wahl5, wahl6])

list1 = wahl8['name'].unique().tolist()
list2 = wahl9['name'].unique().tolist()

inter2 = set(list1).intersection(list2)

wahl9 = wahl9[~wahl9['name'].isin(inter2)]

wahl10 = wahl8.append(wahl9)

vc = wahl10.name.value_counts()
vc = vc[vc>1].index.tolist()


wahl10.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\wahl10.csv')

In [ ]:
wahlfinal = pd.read_excel('wahlfinal' + '.xlsx', 'Sheet1')
print(wahlfinal.shape)

vc = wahlfinal.name.value_counts()
vc = vc[vc>1].index.tolist()
print(vc)

In [ ]:
panelfinal = pd.read_excel('rpaneltotal' + '.xlsx', 'Sheet1')

panelfinal['i']=pd.Series(np.arange(panelfinal['Gemeinde'].nunique()*5) // 5, index=panelfinal.index)
print(panelfinal)

panelfinal.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpanelfinal2.csv')


In [22]:
socialfinal = pd.read_excel('socialfinal' + '.xlsx', 'Sheet1')
rpanelfinal = pd.read_excel('rpaneltotal' + '.xlsx', 'Sheet1')

socialfinal = socialfinal[socialfinal['agglvl'] == "stadtkreise"]
filterr = socialfinal['name'].unique()

rpanelfinal['Gemeinde'] = rpanelfinal['Gemeinde'].str.upper()
rpanelfinal['Gemeinde'] = rpanelfinal["Gemeinde"].str.replace("Ü", "UE")
rpanelfinal['Gemeinde'] = rpanelfinal["Gemeinde"].str.replace("Ö", "OE")
rpanelfinal['Gemeinde'] = rpanelfinal["Gemeinde"].str.replace("Ä", "AE")

def g(row):
    if pd.np.isin(row['Gemeinde'], filterr) == True:
        val = 1
    else:
        val = 0
    return val

rpanelfinal['com'] = rpanelfinal.apply(g, axis=1)
rpanelfinal = rpanelfinal[rpanelfinal['com'] == 1]



print(rpanelfinal['Gemeinde'].nunique())
print(socialfinal['name'].nunique())

socialfinal.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\socialfinal2.csv')
rpanelfinal.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Unemployment data\Unemployment data\rpanelfinal2.csv')

189
189
